In [1]:
import random
import torch
import numpy as np
import matplotlib.pyplot as plt
from model.resnet import ResNet18
from model.smooth_cross_entropy import smooth_crossentropy
from data.cifar import Cifar
from utility.log import Log
from utility.step_lr import StepLR
from utility.bypass_bn import enable_running_stats, disable_running_stats

import sys; sys.path.append("..")
from sam import SAM

In [2]:
adaptive =  True
batch_size = 128
depth = 16
dropout = 0.0
epochs = 200
label_smoothing = 0.1
learning_rate = 0.001
momentum= 0.9
threads = 5
rho = 2.0
weight_decay = 0.0005
width_factor = 8

In [3]:
def initialize(seed: int):
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    torch.backends.cudnn.enabled = True
    torch.backends.cudnn.benchmark = True
    torch.backends.cudnn.deterministic = False

In [4]:
initialize(seed=42)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [5]:
dataset = Cifar(batch_size, threads)
log = Log(log_each=10)
model = ResNet18(num_classes=10).to(device)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


In [6]:
base_optimizer = torch.optim.SGD
optimizer = SAM(model.parameters(), base_optimizer, rho=rho, adaptive=adaptive, lr=learning_rate, weight_decay=weight_decay, momentum=momentum)
scheduler = StepLR(optimizer, learning_rate, epochs)

In [ ]:
train_losses = np.zeros(epochs)
train_accuracy = np.zeros(epochs)
val_losses = np.zeros(epochs)
val_accuracy = np.zeros(epochs)

In [7]:
for epoch in range(epochs):
    model.train()
    log.train(len_dataset=len(dataset.train))

    for batch in dataset.train:
        inputs, targets = (b.to(device) for b in batch)

        # first forward-backward step
        enable_running_stats(model)
        predictions = model(inputs)
        loss = smooth_crossentropy(predictions, targets, smoothing=label_smoothing)
        loss.mean().backward()
        optimizer.first_step(zero_grad=True)

        # second forward-backward step
        disable_running_stats(model)
        smooth_crossentropy(model(inputs), targets, smoothing=label_smoothing).mean().backward()
        optimizer.second_step(zero_grad=True)

        with torch.no_grad():
            correct = torch.argmax(predictions.data, 1) == targets
            log(model, loss.cpu(), correct.cpu(), scheduler.lr())
            scheduler(epoch)

    model.eval()
    train_losses[epoch], train_accuracy[epoch] = log.eval(len_dataset=len(dataset.test))

    with torch.no_grad():
        for batch in dataset.test:
            inputs, targets = (b.to(device) for b in batch)

            predictions = model(inputs)
            loss = smooth_crossentropy(predictions, targets)
            correct = torch.argmax(predictions, 1) == targets
            log(model, loss.cpu(), correct.cpu())
            if epoch > 0:
                val_losses[epoch-1] = log.current_valid_loss
                val_accuracy[epoch-1] = log.current_valid_accuracy

val_losses[epochs-1] = log.current_valid_loss
val_accuracy[epochs-1] = log.current_valid_accuracy

log.flush()

┏━━━━━━━━━━━━━━┳━━━━━━━╸T╺╸R╺╸A╺╸I╺╸N╺━━━━━━━┳━━━━━━━╸S╺╸T╺╸A╺╸T╺╸S╺━━━━━━━┳━━━━━━━╸V╺╸A╺╸L╺╸I╺╸D╺━━━━━━━┓
┃              ┃              ╷              ┃              ╷              ┃              ╷              ┃
┃       epoch  ┃        loss  │    accuracy  ┃        l.r.  │     elapsed  ┃        loss  │    accuracy  ┃
┠──────────────╂──────────────┼──────────────╂──────────────┼──────────────╂──────────────┼──────────────┨
┃           0  ┃      1.3440  │     34.53 %  ┃   1.000e-03  │   03:46 min  ┠┈███████████▓┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┈┨

RuntimeError: Caught RuntimeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "C:\Users\enosh\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\data\_utils\worker.py", line 287, in _worker_loop
    data = fetcher.fetch(index)
  File "C:\Users\enosh\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\data\_utils\fetch.py", line 52, in fetch
    return self.collate_fn(data)
  File "C:\Users\enosh\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\data\_utils\collate.py", line 172, in default_collate
    return [default_collate(samples) for samples in transposed]  # Backwards compatibility.
  File "C:\Users\enosh\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\utils\data\_utils\collate.py", line 172, in <listcomp>
    return [default_collate(samples) for samples in transposed]  # Backwards compatibility.
  File "C:\Users\enosh\AppData\Local\Programs\Python\Python310\lib\site-pac

In [ ]:
torch.save(model.state_dict(), 'sam_model_sgd_resnet.ckpt')

In [ ]:
plt.plot(range(1, epochs + 1), train_losses, label='Train loss')
plt.plot(range(1, epochs + 1), val_losses, label='Validation loss')

In [ ]:
plt.plot(range(1, epochs + 1), train_accuracy, label='Train Accuracy')
plt.plot(range(1, epochs + 1), val_accuracy, label='Train Accuracy')